In [3]:
#import pandas as pd
import csv
import os
import difflib
from collections import Counter
from nltk import word_tokenize
import sys
import re
from urllib.parse import urlsplit
csv.field_size_limit(sys.maxsize)

from IPython.core.debugger import Tracer; debug_here = Tracer()

def preprocess(intext):
    
    text = intext
    
    # Fix strange sentence splitting issue for sentences
    for m in re.findall(r'[a-z]\.[A-Z]', text):
        rep = '. '.join(m.split('.'))
        text = text.replace(m, rep)
        
    # Fix strange sentence splitting issue for sentence + url
    for m in re.findall(r'[a-z]\.http', text):
        rep = '. '.join(m.split('.'))
        text = text.replace(m, rep)
        
    # Add spaces around pipes for image wiki markup
    text = text.replace('|', ' | ')
        
    # Process URLs--just remove http for tokenizer
    for url in re.findall(urlpat, intext):
        host = urlsplit(url).netloc
        text = text.replace(url, host)
        
    return text

In [7]:
# Calculate diff with counter
art_dirpath = '/home/michael/school/research/wp/wp_articles/article_revisions'
diff_dirpath = '/home/michael/school/research/wp/wp_articles/article_diffs'

urlpat = r'https?:\/\/(?:www\.)?[^ ]*'

for f in os.listdir(art_dirpath)[:1]:
    print(f)
    fpath = os.path.join(art_dirpath, f)
    with open(fpath) as csvfile:
        r = csv.reader(csvfile, delimiter='\t')
        prevc = Counter()
        with open(os.path.join(diff_dirpath, f[:-4]+'_diff.csv'), 'w') as outfile:
            w = csv.writer(outfile)
            w.writerow(['article_name', 'timestamp', 'additions', 'deletions', 'editor', 'edit_comment'])
            for i, row in enumerate(r):
                if i % 1000 == 0:
                    print(i/1000, 'k')
#                 textc = Counter(word_tokenize(row[2]))
                
                text = preprocess(row[2])
        
                textc = Counter(text.split())
                addc = textc - prevc
                adds = ' '.join(list(addc.elements()))
#                 adds = ' '.join(word_tokenize(' '.join(list(addc.elements()))))
                delc = prevc - textc
                dels = ' '.join(list(delc.elements()))
#                 dels = ' '.join(word_tokenize(' '.join(list(delc.elements()))))
                
                if adds or dels:
                    line = row[:2] + [adds, dels] + row[3:]
                    w.writerow(line)
                    prevc = textc
            
        print("Wrote diff file for {:s} in {:s}".format(f, diff_dirpath))

gaza_war.tsv
0.0 k
1.0 k


KeyboardInterrupt: 

In [20]:
# Calculate diff with difflib
art_dirpath = '/home/michael/school/research/wp/wp_articles/article_revisions'
diff_dirpath = '/home/michael/school/research/wp/wp_articles/article_diffs'

for f in os.listdir(art_dirpath)[:1]:
    print(f)
    fpath = os.path.join(art_dirpath, f)
    with open(fpath) as csvfile:
        r = csv.reader(csvfile, delimiter='\t')
        prev = ''
        with open(os.path.join(diff_dirpath, f[:-4]+'_diff.csv'), 'w') as outfile:
            w = csv.writer(outfile)
            w.writerow(['article_name', 'timestamp', 'additions', 'deletions', 'editor', 'edit_comment'])
            for i, row in enumerate(r):
                if i % 100 == 0:
                    print(i/1000, 'k')
                text = row[2]
                s = difflib.SequenceMatcher(lambda x: x in " \t", prev, text)
                adds = ''
                dels = ''
                for item in s.get_opcodes():
                    # Additions
                    if item[0] == 'insert' or item[0] == 'replace':
                        adds += text[item[3]:item[4]]

                    # Deletions
                    if item[0] == 'delete' or item[0] == 'replace':
                        dels += text[item[1]:item[2]]

                w.writerow([row[:2] + [adds, dels] + row[3:]])
                prev = text
            
        print("Wrote diff file for {:s}".format(f))

gaza_war.tsv
0.0 k
0.1 k
0.2 k
0.3 k
0.4 k
0.5 k
0.6 k
0.7 k
0.8 k
0.9 k
1.0 k


KeyboardInterrupt: 